Generate questions for AWS SA exam

In [1]:
import requests
from dotenv import load_dotenv
import os

In [2]:
load_dotenv() 
GENERATIVE_URI = os.environ['GENERATIVE_URI']

In [3]:
TOTAL_QUESTIONS = 65
DOMAINS = [
    {
        "name": "Secure",
        "percent": 30,
        "num": 19
    },
    {
        "name": "Resilient",
        "percent": 26,
        "num": 16
    },
    {
        "name": "High-Performing",
        "percent": 24,
        "num": 15
    },
    {
        "name": "Cost-Optimized",
        "percent": 20,
        "num": 15 
    }
]

In [4]:
#determin how many questions in each domain
def get_questions_each_domain():
    
    for domain in DOMAINS:
        no_quest = int(domain['percent'] * TOTAL_QUESTIONS / 100)
        print(no_quest)
#test
#get_questions_each_domain()

In [9]:
#Send a POST request
def post_request_generative_ai(text_prompt):
    HEADER = {'Content-Type': 'application/json'}
    json_data = {
        "contents": [
            { "parts": [
                #{"text": text_prompt + " Please provide a response in a structured JSON format."}]
                {"text": text_prompt + " Please provide a plain string response."}]
            }
        ]
    }

    try:
        r = requests.post(GENERATIVE_URI, headers=HEADER, json=json_data)
        return r.json()
    except Exception as e:
       print(e)
       return {'error': e}

In [ ]:
def generate_list_of_questions():
    no_of_questions = 0
    for domain in DOMAINS:
        domain['num'] = 1 #testing
        text_prompt = "Generate "+str(domain['num'])+" questions for me to practice AWS certified solution architect - associate exam in the domain: Design "+domain['name']+" architecture. I want questions that focus more on real-world scenarios and problem-solving rather than just definitions of services."
        raw_generated_text = post_request_generative_ai(text_prompt)
        #print(raw_generated_text)
        if 'candidates' in raw_generated_text:
            question = raw_generated_text['candidates'][0]['content']['parts'][0]['text']
            print(question)
            no_of_questions += 1
    print(no_of_questions)
#test
generate_list_of_questions()

{'candidates': [{'content': {'parts': [{'text': "Your company's e-commerce platform, hosted on AWS, has experienced a recent surge in fraudulent credit card transactions originating from a specific geographic region.  Design a secure architecture incorporating multiple AWS services to mitigate this risk, focusing on real-time detection and prevention of future fraudulent transactions, while minimizing impact on legitimate users.\n"}], 'role': 'model'}, 'finishReason': 'STOP', 'avgLogprobs': -0.20621844071608322}], 'usageMetadata': {'promptTokenCount': 52, 'candidatesTokenCount': 65, 'totalTokenCount': 117, 'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 52}], 'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 65}]}, 'modelVersion': 'gemini-1.5-flash'}


TypeError: string indices must be integers